In [3]:
import sys
print(sys.version)
import torch

In [4]:
from unetr.utilsUnetr.transforms import CropBedd, RandCropByPosNegLabeld, ResizeOrDoNothingd
from functools import partial
from monai.inferers import sliding_window_inference
from unetr.networks.unetr import UNETR
import os
from unetr.model_module import SegmentationTask
from monai.transforms import LoadImage
import numpy as np
import matplotlib.pyplot as plt
import time 
import torch
from torchvision.utils import save_image
import cv2
import monai.transforms as transforms
import scipy
import os
import glob
import warnings
warnings.filterwarnings("ignore")
from monai import transforms
from monai.transforms import LoadImage
import nibabel as nib
import numpy as np
from pydicom import dcmread
import pandas as pd
import tqdm
import rt_utils
import pydicom
from os import listdir
import tqdm
import json 
from datetime import datetime
import matplotlib.pyplot as plt

In [5]:
def diceSpecSens(maskGT, maskPred):
    predZone=np.where(maskPred==1.0, maskPred,10000)
    gtZone=np.where(maskGT==1.0, maskGT,10000)
    invPredZone=np.where(maskPred!=1.0, maskPred,10000)
    
    vp=np.where(gtZone==maskPred,1,0)
    fn=np.where(maskPred-maskGT==-1,1,0)
    fp=np.where(maskPred-maskGT==1,1,0)
    vn=np.where(invPredZone==maskGT,1,0)
    
    print(np.sum(vp), np.sum(maskPred), np.sum(maskGT))
    return (2*np.sum(vp))/(np.sum(maskPred)+np.sum(maskGT)), (np.sum(vn)/(np.sum(vn)+np.sum(fp))), (np.sum(vp)/(np.sum(vp)+np.sum(fn)))

In [6]:
def transformation():
    dtype= torch.float32
    voxel_space =(1.5, 1.5, 2.0)
    a_min=-200.0
    a_max=300
    b_min=0.0
    b_max=1.0
    clip=True
    crop_bed_max_number_of_rows_to_remove=0
    crop_bed_max_number_of_cols_to_remove=0
    crop_bed_min_spatial_size=(300, -1, -1)
    enable_fgbg2indices_feature=False
    pos=1.0
    neg=1.0
    num_samples=1
    roi_size=(96, 96, 96)
    random_flip_prob=0.2
    random_90_deg_rotation_prob=0.2
    random_intensity_scale_prob=0.1
    random_intensity_shift_prob=0.1
    val_resize=(-1, -1, 250)

    spacing = transforms.Identity()
    if all([space > 0.0 for space in voxel_space]):
        spacing = transforms.Spacingd(
            keys=["image", "label"], pixdim=voxel_space, mode=("bilinear", "nearest")
        ) # to change the dimension of the voxel to have less data to compute

        posneg_label_croper_kwargs = {
                "keys": ["image", "label"],
                "label_key": "label",
                "spatial_size": roi_size,
                "pos": pos,
                "neg": neg,
                "num_samples": num_samples,
                "image_key": "image",
                "allow_smaller": True,
        }

        fgbg2indices = transforms.Identity()
        if enable_fgbg2indices_feature:
            fgbg2indices = transforms.FgBgToIndicesd(
                    keys=["image", "label"], image_key="label", image_threshold=0.0
            ) # to crop samples close to the label mask
            posneg_label_croper_kwargs["fg_indices_key"] = "image_fg_indices"
            posneg_label_croper_kwargs["bg_indices_key"] = "image_bg_indices"
        else:
            posneg_label_croper_kwargs["image_threshold"] = 0.0

    transform = transforms.Compose(
                [
                    transforms.Orientationd(keys=["image", "label"], axcodes="LAS", allow_missing_keys=True), # to have the same orientation
                    spacing,
                    transforms.ScaleIntensityRanged(
                        keys=["image"], a_min=a_min, a_max=a_max, b_min=b_min, b_max=b_max, clip=clip, allow_missing_keys=True
                    ), # scales image from a values to b values
                    CropBedd(
                        keys=["image", "label"], image_key="image",
                        max_number_of_rows_to_remove=crop_bed_max_number_of_rows_to_remove,
                        max_number_of_cols_to_remove=crop_bed_max_number_of_cols_to_remove,
                        min_spatial_size=crop_bed_min_spatial_size,
                        axcodes_orientation="LAS",
                    ), # crop the bed from the image (useless data)
                    transforms.CropForegroundd(keys=["image", "label"], source_key="image", allow_missing_keys=True), # remove useless background image part
                    fgbg2indices,
                    transforms.RandFlipd(keys=["image", "label"], prob=random_flip_prob, spatial_axis=0, allow_missing_keys=True), # random flip on the X axis
                    transforms.RandFlipd(keys=["image", "label"], prob=random_flip_prob, spatial_axis=1, allow_missing_keys=True), # random flip on the Y axis
                    transforms.RandFlipd(keys=["image", "label"], prob=random_flip_prob, spatial_axis=2, allow_missing_keys=True), # random flip on the Z axis
                    transforms.RandRotate90d(keys=["image", "label"], prob=random_90_deg_rotation_prob, max_k=3, allow_missing_keys=True), # random 90 degree rotation
                    transforms.RandScaleIntensityd(keys="image", factors=0.1, prob=random_intensity_scale_prob), # random intensity scale
                    transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=random_intensity_shift_prob), # random intensity shifting
                    transforms.ToTensord(keys=["image", "label"], dtype=dtype), # to have a PyTorch tensor as output
                ]
            )
    return transform

def loadModel(pathModelFile):
    #map_location = torch.device('cpu') a faire pas ici mais dans unetr/model_module.py ligne 133 ou try faire ailleurs
    model= SegmentationTask.load_from_checkpoint(pathModelFile)
    model.eval()
    return model

def loadDicomImage(slices_folder):
    image=torch.tensor([LoadImage(image_only=True)(slices_folder)])
    image=((image/np.max(np.array(image)))*255)
    return image


def applyTransforms(transform, image):
    print("----------------")
    print(image.ndim)
    print("----------------")
    print("applyT", image.shape)
    image={"image":image, "label":torch.zeros_like(image),"patient_id":'201905984', "has_meta":True}
    image=transform(image)
    print("applyT", image['image'].shape)
    return image

def applyTransforms2(transform, image):
    # Assurez-vous que l'image est un tensor PyTorch
    image = torch.tensor(image, dtype=torch.float32)
    
    image = (image / torch.max(image)) * 255
    if image.ndim == 3:
        image = image.unsqueeze(0)
    print("----------------")
    print(image.ndim)
    print("----------------")
    data = {"image": image, "label": torch.zeros_like(image), "patient_id": '201905984', "has_meta": True}
    transformed = transform(data)
    return transformed


def applyUNETR(dicoImage, model):
    label =sliding_window_inference(inputs=dicoImage["image"][None], 
                                            roi_size=(96, 96, 96), 
                                            sw_batch_size=4,
                                            predictor=model,
                                            overlap=0.5)

    label = torch.argmax(label, dim=1, keepdim=True)
    
    size=label.shape
    print("applyUNETR", size[1], size[2], size[3], size[4])
    dicoImage["label"]=label.reshape((size[1], size[2], size[3], size[4]))
    return dicoImage

def disapplyTransforms(transform, dicoImage):
    dicoImage = transform.inverse(dicoImage)
    return dicoImage["label"], dicoImage["image"]



def getLabelOfIRM_from_path(pathSlicesIRM, pathModelFile):
    image = loadDicomImage(pathSlicesIRM)
    transform = transformation()
    dicoImage = applyTransforms(transform, image)
    model = loadModel(pathModelFile)
    dicoImage = applyUNETR(dicoImage, model)
    label, imageT = disapplyTransforms(transform, dicoImage)
    return image/255, label, imageT

# Pipeline complet pour charger, transformer, segmenter et désappliquer les transformations
def getLabelOfIRM_from_nifti(nifti_image: nib.Nifti1Image, pathModelFile: str):
    transform = transformation()
    transformed_image = applyTransforms2( transform, nifti_image.get_fdata())

    model = loadModel(pathModelFile)
    #dico_image = {"image": transformed_image, "label": torch.zeros_like(transformed_image)}
    dico_image = applyUNETR(transformed_image, model)

    label, imageT = disapplyTransforms(transform, dico_image)
    return nifti_image.get_fdata() / 255, label, imageT

def getLabelOfIRM_from_image(image, mask, pathModelFile):
    image=torch.tensor([image])
    image=((image/np.max(np.array(image)))*255)
    transform = transformation()
    dicoImage = applyTransforms(transform, image)
    model = loadModel(pathModelFile)
    dicoImage = applyUNETR(dicoImage, model)
    label, imageT = disapplyTransforms(transform, dicoImage)
    return image/255, label, imageT

#pathSlicesIRM='/home/aurelien/Documents/Segmentation Métastases cérébrales et méningiomes par IA/UNETR/metastase_IA/BaseDonnée220Patients/201905984/RM'
#pathModelFile="/home/aurelien/Documents/Segmentation Métastases cérébrales et méningiomes par IA/UNETR/metastase_IA/RunAll4/checkpoints/checkpoint-epoch=1599-val_loss=0.225.ckpt"

#image, label, imageT = getLabelOfIRM(pathSlicesIRM, pathModelFile)

In [7]:
def miseEnDossier(patient_id):
    path_newDirRM = os.path.join(meta_data_dir, patient_id, "RM/")
    path_newDirMETA = os.path.join(meta_data_dir, patient_id, "META/")
    if not os.path.exists(path_newDirRM):
        os.mkdir(path_newDirRM)
    if not os.path.exists(path_newDirMETA):
        os.mkdir(path_newDirMETA)
    for mr in glob.glob(os.path.join(meta_data_dir, patient_id, "MR*")):
        path_dir, name_file= os.path.split(mr)
        os.rename(mr, str(path_newDirRM)+str(name_file))
    for rs in glob.glob(os.path.join(meta_data_dir, patient_id, "RS*")):
        path_dir, name_file= os.path.split(rs)
        os.rename(rs, str(path_newDirMETA)+str(name_file))
        
def fusionMask(mask):
        if (type(mask)==list):
            premMask = mask[0]
            for autMask in mask[1:]:
                premMask = np.where(autMask > 0, autMask, premMask)
            return premMask
        else:
            return mask
    
listeName=[]
def load_dicom_with_mask(slices_folder, mask_file):
    rt_struct = rt_utils.RTStructBuilder.create_from(slices_folder, mask_file)
    maskAllAlone=[]
    for i in rt_struct.get_roi_names():
        if "GTV" in i or "recid" in i or "FRONTAL" in i or "GTC" in i or "Cerebelleux" in i or "PARIET" in i or "gtv" in i or "M1." in i or "M2." in i or "FRONTAL" in i:
            if "patient" not in i and "POST OP" not in i and "Préop" not in i and "External" not in i and "cav" not in i and "Cav" not in i and "cavite" not in i and "cavité" not in i and "Cavité" not in i and "PTV" not in i and "ANCIEN" not in i and "Ancien" not in i and "ANC" not in i and "Anecien" not in i and "PTC" not in i:
                maskAllAlone.append(rt_struct.get_roi_mask_by_name(i).astype(np.float32))
                listeName.append(i)
    mask = fusionMask(maskAllAlone)
    img = np.array(LoadImage( image_only=True)(slices_folder))

    img = np.rot90(img)
    img = np.flip(img, 0)
    img = (img/np.max(img))*255
    return img, np.array(mask)


def load_dicom_series_without_mask(patient_folder):
    img = LoadImage( image_only=True)(patient_folder)
    mask = np.zeros_like(img)
    return np.array(img), mask
  
def filter_patients_with_meta(patient_id: str) -> bool:
    miseEnDossier(patient_id)
    # if no folder in patient directory
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "*"))) == 0:
        return False

    # if no named folders "META" or "RM"
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META"))) == 0 or len(glob.glob(os.path.join(meta_data_dir, patient_id, "RM"))) == 0:
        return False
    
    # if no dicom image in the META folder
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META", "*"))) == 0:
        return False
    
    return True

def filter_patients_without_meta(patient_id: str) -> bool:
    miseEnDossier(patient_id)
    # if no folder in patient directory
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "*"))) == 0:
        return False

    # if no named folders "META" or "RM"
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "RM"))) == 0:
        return False

    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META"))) != 0:
        if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META", "*"))) != 0:
            return False
    return True

In [8]:
from typing import List
import pydicom
from io import BytesIO
import os 
import nibabel as nib
import numpy as np
from monai.transforms import Compose, Orientationd, ScaleIntensityRanged, CropForegroundd, ToTensord
from monai.transforms import RandFlipd, RandRotate90d, RandScaleIntensityd, RandShiftIntensityd
import torch
from monai.inferers import sliding_window_inference
from unetr.model_module import SegmentationTask

# fonction de chargement de dicoms en mémoire pour le test de conversion en nifti
def load_dicom_files_from_directory(directory_path):
    dicom_datasets = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".dcm"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'rb') as file:
                dicom_file = pydicom.dcmread(BytesIO(file.read()))
                dicom_datasets.append(dicom_file)
    return dicom_datasets


# chargement de données médicales dicoms en mémoire depuis un répertoire de l'ordinateur : 

pathSlicesIRM = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM'

# lecture des fichiers dicom avec pydicom

dicom_datasets = load_dicom_files_from_directory(pathSlicesIRM)

#################### API #############################################################################


def dicom_to_nifti_in_memory(dicom_datasets: List[pydicom.Dataset]) -> nib.Nifti1Image:
    image_slices = [ds.pixel_array for ds in dicom_datasets]
    volume_3d = np.stack(image_slices, axis=-1)
    affine = np.eye(4) # necessaire pour les algorithmes de traitement d'images médicales, ça permet de savoir comment les voxels sont disposés dans l'espace
    nifti_image = nib.Nifti1Image(volume_3d, affine)
    return nifti_image


niftis =  dicom_to_nifti_in_memory(dicom_datasets)

In [9]:
# Chemins vers les dossiers d'images DICOM et le modèle UNETR
pathSlicesIRM = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM'
pathSlicesIRM2 = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/nifti_s/201704321/image.nii.gz'
pathModelFile = '/Users/romain/Downloads/Modeles_Pre_Entraines/checkpoint_epoch1599_val_loss0255.cpkt'

# Charger les étiquettes pour les images DICOM
#image, label, imageT = getLabelOfIRM_from_path(pathSlicesIRM2, pathModelFile)
image, label, imageT = getLabelOfIRM_from_nifti(niftis, pathModelFile)

# Afficher les résultats
print("Image shape:", image.shape)
print("Label shape:", label.shape)
print("Transformed image shape:", imageT.shape)